### Import 

In [1]:
import pandas as pd
from nltk.corpus import stopwords 
import string
from nltk.stem.wordnet import WordNetLemmatizer
from nltk import word_tokenize 
import re
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
from sentence_transformers import SentenceTransformer

In [3]:
model_name = 'bert-base-nli-mean-tokens'

In [4]:
model = SentenceTransformer(model_name)

Downloading: 100%|██████████████████████████████████████████████████████████████████████| 190/190 [00:00<00:00, 205kB/s]
Downloading: 100%|█████████████████████████████████████████████████████████████████| 3.95k/3.95k [00:00<00:00, 3.91MB/s]
Downloading: 100%|█████████████████████████████████████████████████████████████████████| 2.00/2.00 [00:00<00:00, 559B/s]
Downloading: 100%|█████████████████████████████████████████████████████████████████████| 122/122 [00:00<00:00, 72.9kB/s]
Downloading: 100%|███████████████████████████████████████████████████████████████████| 438M/438M [00:35<00:00, 12.3MB/s]
Downloading: 100%|███████████████████████████████████████████████████████████████████| 53.0/53.0 [00:00<00:00, 26.7kB/s]
Downloading: 100%|█████████████████████████████████████████████████████████████████████| 112/112 [00:00<00:00, 57.9kB/s]
Downloading: 100%|███████████████████████████████████████████████████████████████████| 466k/466k [00:00<00:00, 1.13MB/s]
Downloading: 100%|██████████████

### Load Data

In [5]:
pd.set_option('display.max_columns', None)

In [6]:
df = pd.read_csv("../raw_data/clean_data.csv")

In [7]:
df.head()

,Unnamed: 0,age,status,sex,orientation,body_type,diet,drinks,drugs,education,income,job,offspring,pets,smokes,speaks,religion_info,strict,speaks_cleaned,primary_language,number_of_languages,essay0_cleaned,essay1_cleaned,essay2_cleaned,essay3_cleaned,essay4_cleaned,essay5_cleaned,essay6_cleaned,essay7_cleaned,essay8_cleaned,essay9_cleaned
0,0,22,single,m,straight,a little extra,no restriction,socially,never,working on college/university,104395,transportation,might want kids,likes dogs and likes cats,sometimes,english,agnosticism,0,['english'],english,1,would love think kind intellectual either dumb...,currently working international agent freight ...,making people laugh ranting good salting findi...,way look six foot half asian half caucasian mu...,book absurdistan republic mouse men book made ...,food water cell phone shelter,duality humorous thing,trying find someone hang anything except club,new california looking someone wisper secret,want swept foot tired norm want catch coffee b...
1,1,35,single,m,straight,average,no restriction,often,sometimes,working on space camp,80000,hospitality / travel,might want kids,likes dogs and likes cats,no,"english (fluently), spanish (poorly), french (...",agnosticism,0,"['english', 'fluently', 'spanish', 'poorly', '...",english,4,chef mean workaholic love cook regardless whet...,dedicating everyday unbelievable badass,silly ridiculous amonts fun wherever smart as ...,NaN,die hard christopher moore fan really watch lo...,delicious porkness glory big as doughboy sinki...,NaN,NaN,open share anything,NaN
2,2,38,available,m,straight,thin,no restriction,socially,rather not say,graduated from masters program,104395,other,rather not say,likes dogs and likes cats,no,"english, french, c++",agnosticism,0,"['english', 'french', 'c']",english,3,ashamed much writing public text online dating...,make nerdy software musician artist experiment...,improvising different context alternating pres...,large jaw large glass physical thing people co...,okay cultural matrix get specific like crossha...,movement conversation creation contemplation t...,NaN,viewing listening dancing talking drinking per...,five year old known boogerman,bright open intense silly ironic critical cari...
3,3,23,single,m,straight,thin,veggie,socially,rather not say,working on college/university,20000,student,doesn't want kids,likes dogs and likes cats,no,"english, german (poorly)",agnosticism,0,"['english', 'german', 'poorly']",english,2,work library go school,reading thing written old dead people,playing synthesizer organizing book according ...,socially awkward best,bataille celine beckett lynch jarmusch r w fas...,NaN,cat german philosophy,NaN,NaN,feel inclined
4,4,29,single,m,straight,athletic,no restriction,socially,never,graduated from college/university,104395,artistic / musical / writer,rather not say,likes dogs and likes cats,no,english,agnosticism,0,['english'],english,1,hey going currently vague profile know come so...,work work work work play,creating imagery look http bagsbrown blogspot ...,smile lot inquisitive nature,music band rapper musician moment thee oh see ...,NaN,NaN,NaN,NaN,NaN


In [11]:
sentence_vecs = model.encode(df.essay0_cleaned[0:2])

In [12]:
sentence_vecs.shape

(2, 768)

In [13]:
from sklearn.metrics.pairwise import cosine_similarity

In [14]:
cosine_similarity(
    [sentence_vecs[0]],
    sentence_vecs[1:]
)

array([[0.7325585]], dtype=float32)

In [15]:
df.essay0_cleaned[0]

'would love think kind intellectual either dumbest smart guy smartest dumb guy say tell difference love talk idea concept forge odd metaphor instead reciting cliche like simularities friend mine house underwater salt mine favorite word salt way weird choice know thing life better metaphor seek make little better everyday productively lazy way got tired tying shoe considered hiring five year old would probably tie shoe decided wear leather shoe dress shoe love really serious really deep conversation really silly stuff willing snap light hearted rant kiss funny able make laugh able bend spoon mind telepathically make smile still work love life cool letting wind blow extra point reading guessing favorite video game hint given yet lastly good attention span'

In [16]:
df.essay0_cleaned[1]

'chef mean workaholic love cook regardless whether work love drink eat food probably really bad love around people resemble line love outdoors avid skier snowing tahoe least confident friendly interested acting typical guy time patience rediculous act territorial pissing overall likable easygoing individual adventurous always looking forward new thing hopefully sharing right person'

In [ ]:
dfok.head(20)

In [ ]:
df.education = df.education.fillna('graduated from college/university')

In [ ]:
df.head(20)

In [ ]:
df.

In [ ]:
dfok.head(20)

In [ ]:
df1 = pd.read_csv("../raw_data/clean_data_0.csv")
df2 = pd.read_csv("../raw_data/clean_data_1.csv")
df3 = pd.read_csv("../raw_data/clean_data_2.csv")

In [ ]:
df = df1.append(df2).append(df3)

In [ ]:
df.rename(columns={"Unnamed: 0": " "}, errors="raise",inplace=True)

In [ ]:
df.set_index(" ")

In [ ]:
df.education.value_counts()

In [ ]:
df

In [ ]:
df.head

In [ ]:
df.tail()

In [ ]:
df.pets.value_counts()

In [ ]:
df.pets=df.pets.fillna('likes dogs and likes cats')

df.pets.replace(['likes dogs','likes dogs and has cats','has dogs','has dogs and likes cats','has dogs and has cats','has cats','likes cats'],'likes dogs and likes cats',inplace=True)

df.pets.replace(['has dogs and dislikes cats','dislikes cats'],'likes dogs and dislikes cats',inplace=True)

df.pets.replace(['dislikes dogs and has cats','dislikes dogs'],'dislikes dogs and likes cats',inplace=True)

In [ ]:
df.head()

### My topics to clean

essay8- The most private thing I am willing to admit

essay9- You should message me if...

status, diet, ethnicity, last_online, religion

we have decided to remove last_online

#### status

In [ ]:
df.status.replace("unknown","available",inplace=True)

In [ ]:
df['status'] = np.where(df['status'].str.contains('single'), 'available', df['status'])
df = pd.concat([pd.get_dummies(df.status, prefix='status', prefix_sep='_'), df], axis = 1)

df.status.value_counts()

In [ ]:
df.head(30)

In [ ]:
df['status'] = df['status'].map({'unknown': 'available', 'male': 0})

#### diet

In [ ]:
df.diet.fillna("no restriction", inplace=True)

# create dummy variable: strict (1=strictly following a diet)
df['strict'] = 0
df.loc[df.diet.str.contains('strictly'), 'strict'] = 1
df.loc[df.diet.str.len()==1, 'strict'] = 1

# group diets
df['diet'] = np.where(df['diet'].str.contains('strictly anything|mostly other|anything|mostly anything|strictly other|other'), 'no restriction', df['diet'])
df.loc[df.diet=='no restriction', 'strict'] = 0
df['diet'] = np.where(df['diet'].str.contains('mostly vegetarian|strictly vegan|strictly vegetarian|mostly vegan|vegan|vegetarian'), 'veggie', df['diet'])
df['diet'] = np.where(df['diet'].str.contains('mostly kosher|strictly kosher|kosher'), 'kosher', df['diet'])
df['diet'] = np.where(df['diet'].str.contains('mostly halal|strictly halal|halal'), 'halal', df['diet'])

In [ ]:
df[df["status"]=="unknown"]

In [ ]:
df[df["diet"]=="veggie"]

In [ ]:
df.diet.unique()

#### ethnicity

We can split ethnicity into 6 big groups

white,asian,hispanic/latin,black,middle eastern

In [ ]:
df['eth_num'] = df.ethnicity.str.len()
df["ethnicity2"] = "race"
df.loc[df.eth_num<2, "ethnicity2"] = df.ethnicity.str[0]
df.loc[df.eth_num==2, "ethnicity2"] = "biracial"
df.loc[df.eth_num>2, "ethnicity2"] = "multiracial"
df.loc[df.eth_num>2, "eth_num"] = 3

In [ ]:
df.head()

#### religion

In [ ]:
(df.religion.isnull().sum()/len(df))*100

In [ ]:
df["religion"] = df["religion"].fillna('agnostic')

In [ ]:
df.religion.isnull().sum()

In [ ]:
pd.set_option('display.max_columns', None)

##### we will have to try different methods of fill NA in our model
such as fill NA as agnostic or other

#### we can also try to split the data in different ways such as
shrink into 6 categories or 
split into categories and add column for intesity 


In [ ]:
df.religion.value_counts()

##### fill NA with agnostic



In [ ]:
dfagn = df.copy()

dfagn["religion"] = df["religion"].fillna("agnosticism")

##### fill NA with other

In [ ]:
dfother = df.copy()
dfother["religion"] = df["religion"].fillna("other")

##### add features for seriousness

In [ ]:
dfagn['rel_scale'] = np.where(dfagn['religion'].str.contains("very serious"), 1, 0)

In [ ]:
def create_religion_scale(df):
    conditions = [
        df['religion'].str.contains("very serious"),
        df['religion'].str.contains("somewhat serious"),
        df['religion'].str.contains("not too serious"),
        df['religion'].str.contains("laughing")]
    choices = ['very serious', 'somewhat serious', 'not too serious','laughing']
    df['rel_scale'] = np.select(conditions, choices, default='normal')
    return df

In [ ]:
df.religion.fillna("other", inplace=True)

# create 1 variable: serious (1=yes, 0=neutral, -1=no)
df["serious"] = 0
df.loc[df.religion.str.contains("very|somewhat"), "serious"] = 1
df.loc[df.religion.str.contains("laughing"), "serious"] = -1
df.religion = df.religion.str.split().str[0]
# df.groupby("religion")["serious"].mean()

# seriousness by religion
plt.plot(df.groupby("religion")["serious"].mean(), marker='o', color="#5e3a98")
plt.xticks(rotation=45)
plt.title("how seriously does a user take religion?")
plt.ylabel("average seriousness")
plt.show()


In [ ]:
dfagn = create_religion_scale(dfagn)
dfagn

In [ ]:
dfother = create_religion_scale(dfother)

dfother

#### cleaning function

In [ ]:
def clean (text):
    for punctuation in string.punctuation:
        text = text.replace(punctuation, ' ') # Remove Punctuation
    lowercased = text.lower() # Lower Case
    tokenized = word_tokenize(lowercased) # Tokenize
    words_only = [word for word in tokenized if word.isalpha()] # Remove numbers
    stop_words = set(stopwords.words('english')) # Make stopword list
    without_stopwords = [word for word in words_only if not word in stop_words] # Remove Stop Words
    lemma=WordNetLemmatizer() # Initiate Lemmatizer
    lemmatized = [lemma.lemmatize(word) for word in without_stopwords] # Lemmatize
    return lemmatized


In [ ]:
df['essay0']=df['essay8'].fillna('')
df['essay1']=df['essay9'].fillna('')
df['essay2']=df['essay8'].fillna('')
df['essay3']=df['essay9'].fillna('')
df['essay4']=df['essay8'].fillna('')
df['essay5']=df['essay9'].fillna('')
df['essay6']=df['essay8'].fillna('')
df['essay7']=df['essay9'].fillna('')
df['essay8']=df['essay8'].fillna('')
df['essay9']=df['essay9'].fillna('')


In [ ]:
df['essay0clean'] = df['essay0'].apply(lambda x: clean(x)).apply(lambda x:' '.join(x))
df['essay1clean'] = df['essay1'].apply(lambda x: clean(x)).apply(lambda x:' '.join(x))
df['essay2clean'] = df['essay2'].apply(lambda x: clean(x)).apply(lambda x:' '.join(x))
df['essay3clean'] = df['essay3'].apply(lambda x: clean(x)).apply(lambda x:' '.join(x))
df['essay4clean'] = df['essay4'].apply(lambda x: clean(x)).apply(lambda x:' '.join(x))
df['essay5clean'] = df['essay5'].apply(lambda x: clean(x)).apply(lambda x:' '.join(x))
df['essay6clean'] = df['essay6'].apply(lambda x: clean(x)).apply(lambda x:' '.join(x))
df['essay7clean'] = df['essay7'].apply(lambda x: clean(x)).apply(lambda x:' '.join(x))
df['essay8clean'] = df['essay8'].apply(lambda x: clean(x)).apply(lambda x:' '.join(x))
df['essay9clean'] = df['essay9'].apply(lambda x: clean(x)).apply(lambda x:' '.join(x))

In [ ]:
dfessay = df[['essay0clean','essay1clean','essay2clean','essay3clean','essay4clean','essay5clean','essay6clean','essay7clean','essay8clean','essay9clean']].copy()

In [ ]:
dfessay.head()

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
vectorizer = TfidfVectorizer(max_df=0.5,ngram_range=(3,3))

data_vectorized = vectorizer.fit_transform(dfessay["essay0clean"])

In [ ]:
from sklearn.manifold import TSNE

tsne = TSNE(n_components=2,perplexity=40,random_state=42)
tsne_essays = tsne.fit_transform(data_vectorized)

In [ ]:
plt.scatter(tsn)

In [ ]:
df['essay8clean'][0]

In [ ]:
df['essay8'][0]

In [ ]:
df['essay9clean'][0]

In [ ]:
df['essay9'][0]

In [4]:
import os
import sys
nb_dir = os.path.split(os.getcwd())[0]
if nb_dir not in sys.path:
    sys.path.append(nb_dir)

In [5]:
from RecommenDate import clean_data

[nltk_data] Downloading package punkt to /home/rohito/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/rohito/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/rohito/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/rohito/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [6]:
clean_df = clean_data.clean_data(dfok)

In [7]:
clean_df.head()

,age,status,sex,orientation,body_type,diet,drinks,drugs,education,income,job,offspring,pets,smokes,speaks,religion_info,strict,speaks_cleaned,primary_language,number_of_languages,essay0_cleaned,essay1_cleaned,essay2_cleaned,essay3_cleaned,essay4_cleaned,essay5_cleaned,essay6_cleaned,essay7_cleaned,essay8_cleaned,essay9_cleaned
0,22,single,m,straight,a little extra,no restriction,socially,never,working on college/university,104395,transportation,might want kids,likes dogs and likes cats,sometimes,english,agnosticism,0,[english],english,1,would love think kind intellectual either dumb...,currently working international agent freight ...,making people laugh ranting good salting findi...,way look six foot half asian half caucasian mu...,book absurdistan republic mouse men book made ...,food water cell phone shelter,duality humorous thing,trying find someone hang anything except club,new california looking someone wisper secret,want swept foot tired norm want catch coffee b...
1,35,single,m,straight,average,no restriction,often,sometimes,working on space camp,80000,hospitality / travel,might want kids,likes dogs and likes cats,no,"english (fluently), spanish (poorly), french (...",agnosticism,0,"[english, fluently, spanish, poorly, french, p...",english,4,chef mean workaholic love cook regardless whet...,dedicating everyday unbelievable badass,silly ridiculous amonts fun wherever smart as ...,,die hard christopher moore fan really watch lo...,delicious porkness glory big as doughboy sinki...,,,open share anything,
2,38,available,m,straight,thin,no restriction,socially,rather not say,graduated from masters program,104395,other,rather not say,likes dogs and likes cats,no,"english, french, c++",agnosticism,0,"[english, french, c]",english,3,ashamed much writing public text online dating...,make nerdy software musician artist experiment...,improvising different context alternating pres...,large jaw large glass physical thing people co...,okay cultural matrix get specific like crossha...,movement conversation creation contemplation t...,,viewing listening dancing talking drinking per...,five year old known boogerman,bright open intense silly ironic critical cari...
3,23,single,m,straight,thin,veggie,socially,rather not say,working on college/university,20000,student,doesn't want kids,likes dogs and likes cats,no,"english, german (poorly)",agnosticism,0,"[english, german, poorly]",english,2,work library go school,reading thing written old dead people,playing synthesizer organizing book according ...,socially awkward best,bataille celine beckett lynch jarmusch r w fas...,,cat german philosophy,,,feel inclined
4,29,single,m,straight,athletic,no restriction,socially,never,graduated from college/university,104395,artistic / musical / writer,rather not say,likes dogs and likes cats,no,english,agnosticism,0,[english],english,1,hey going currently vague profile know come so...,work work work work play,creating imagery look http bagsbrown blogspot ...,smile lot inquisitive nature,music band rapper musician moment thee oh see ...,,,,,


In [8]:
clean_df.offspring.value_counts()

rather not say       35561
might want kids      15727
doesn't want kids     4776
wants more kids       3882
Name: offspring, dtype: int64

In [ ]:
clean_df.offspring.replace(["doesn't have kids, and doesn't want any","has kids, but doesn't want more","has a kid, but doesn't want more"],"doesn't want kids",inplace=True)
clean_df.offspring.replace(["doesn't have kids","doesn't have kids, but might want them","has kids","has a kid","has a kid, and might want more","might want kids","has kids, and might want more"],"might want kids",inplace=True)
clean_df.offspring.replace(["doesn't have kids, but wants them","wants kids","has a kid, and wants more","has kids, and wants more"],"wants more kids",inplace=True)

In [ ]:
clean_df.offspring.value_counts()

In [9]:
ls ..

Makefile     notebooks/  README.md      requirements.txt  setup.py
MANIFEST.in  raw_data/   RecommenDate/  scripts/          tests/


In [10]:
from pathlib import Path  
filepath = Path('../raw_data/clean_data.csv')  
filepath.parent.mkdir(parents=True, exist_ok=True)  
clean_df.to_csv(filepath) 